# Подход 1: градиентный бустинг "в лоб"

## Загрузка обучающей и тестовой выборок

In [208]:
import pandas as pd

features = pd.read_csv('./features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [209]:
features_test = pd.read_csv('./features_test.csv', index_col='match_id')

features_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


## Найдем признаки, отсутствующие в тестовой выборке.

In [210]:
itogs = [x for x in features.columns if x not in features_test.columns]

itogs

['duration',
 'radiant_win',
 'tower_status_radiant',
 'tower_status_dire',
 'barracks_status_radiant',
 'barracks_status_dire']

## Целевой признак - 'radiant_win'

In [211]:
y = features['radiant_win'].as_matrix()

y

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

## Оставим только те признаки, которые есть в тестовой выборке

In [212]:
features = features[[x for x in features.columns if x in features_test.columns]]

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


## Найдем все признаки в которых присутствуют пропуски

In [213]:
[k for k,v in dict(features.count() / features.shape[0]).items() if v < 1]

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

## Возможная причина пропуска в данных:

### Признак first_blood_time - игровое время первой крови.

#### Первая кровь не была пролита в течение первых 5 минут матча. Возможно обе команды были настроены на пассивную игру и попытки нарастить преимущество в лейте.

### Признак radiant_first_ward_time - время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля

#### Ни один из игроков команды dire не покупал "ward" в течение первых 5 минут матча. Игра без саппортов на низком рейтинге...

## Заменяем пропуски в обучающей выборке на 0 и проверяем, что не осталось ни одного признака с пропусками.

In [214]:
features = features.fillna(0)

[k for k,v in dict(features.count() / features.shape[0]).items() if v < 1]

[]

## Обучение модели градиентного бустинга

In [215]:
from sklearn.model_selection import KFold, cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)

X = features.as_matrix()

In [216]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

In [183]:
for n in [10,20,30,40,50,60, 70, 80, 90, 100]:
    
    start_time = datetime.datetime.now()
    
    gbc = GradientBoostingClassifier(n_estimators=n, verbose=False, random_state=42)
    
    gbc.fit(X, y)
    
    auc = []
    
    cv_time = datetime.datetime.now()
    
    for train_index, test_index in kf.split(X):
    
        X_train = X[train_index]

        y_train = y[train_index]

        pred1 = gbc.predict_proba(X_train)[:, 1]

        auc1 = roc_auc_score(y_train, pred1)
        
        auc.append(auc1)
        
    print("Деревьев: ", n, " Среднее значение AUC: ", round(np.mean(auc),4), " Время на кросс-валидацию: ", datetime.datetime.now() - cv_time, " Затраченное время: ", datetime.datetime.now() - start_time)

Деревьев:  10  Среднее значение AUC:  0.6725  Время на кросс-валидацию:  0:00:00.598596  Затраченное время:  0:00:05.186569
Деревьев:  20  Среднее значение AUC:  0.6916  Время на кросс-валидацию:  0:00:00.691316  Затраченное время:  0:00:08.900033
Деревьев:  30  Среднее значение AUC:  0.7002  Время на кросс-валидацию:  0:00:00.942740  Затраченное время:  0:00:13.402880
Деревьев:  40  Среднее значение AUC:  0.7056  Время на кросс-валидацию:  0:00:00.932483  Затраченное время:  0:00:16.918338
Деревьев:  50  Среднее значение AUC:  0.7102  Время на кросс-валидацию:  0:00:01.507147  Затраченное время:  0:00:26.341906
Деревьев:  60  Среднее значение AUC:  0.7138  Время на кросс-валидацию:  0:00:01.490080  Затраченное время:  0:00:32.759553
Деревьев:  70  Среднее значение AUC:  0.7167  Время на кросс-валидацию:  0:00:01.916687  Затраченное время:  0:00:41.496358
Деревьев:  80  Среднее значение AUC:  0.7192  Время на кросс-валидацию:  0:00:01.919636  Затраченное время:  0:00:47.628832
Деревьев

In [ ]:
for n in [10,20,30,40,50,60, 70, 80, 90, 100]:
    
    start_time = datetime.datetime.now()
    
    gbc = GradientBoostingClassifier(n_estimators=n, verbose=False, random_state=42)
    
    gbc.fit(X, y)
    
    cv_time = datetime.datetime.now()
    
    model_scores = cross_val_score(gbc, X, y, cv=kf, scoring='roc_auc')
        
    print("Деревьев: ", n, " Среднее значение AUC: ", round(np.mean(model_scores),4), " Время на кросс-валидацию: ", datetime.datetime.now() - cv_time, " Затраченное время: ", datetime.datetime.now() - start_time)

Деревьев:  10  Среднее значение AUC:  0.6649  Время на кросс-валидацию:  0:00:35.030323  Затраченное время:  0:00:40.225922
Деревьев:  20  Среднее значение AUC:  0.6825  Время на кросс-валидацию:  0:01:09.148903  Затраченное время:  0:01:18.846723
Деревьев:  30  Среднее значение AUC:  0.69  Время на кросс-валидацию:  0:01:38.813180  Затраченное время:  0:01:53.512039
Деревьев:  40  Среднее значение AUC:  0.694  Время на кросс-валидацию:  0:02:06.278874  Затраченное время:  0:02:27.321037


## С ростом числа деревьев, при сохранении остальных параметров увеличивается общее время расчета. Среднее значение AUC по кросс-валидации увеличивается при росте числа деревьев, что говорит нам о том, что использование большего числа деревьев увеличивает качество модели на обучающей выборке. Однако, с ростом качества на обучении увеличивается риск переобучения

# Подход 2: логистическая регрессия

In [197]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()

In [193]:
X = scaler.fit_transform(features.as_matrix())

In [196]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,-2.544364,1.540688,-1.244228,1.400808,1.525972,0.734957,0.969743,-0.537757,-0.578083,-0.509023,...,-0.987486,1.066448,-0.041743,-0.262922,0.640648,0.018054,0.562864,-0.551154,1.846004,-1.121494
1,-2.540452,-0.927798,-0.292258,0.501314,-0.080139,-0.247570,-0.246859,-0.537757,1.017574,1.492930,...,-0.987486,-0.338591,0.578946,-0.262922,0.379585,1.066668,0.562864,0.678170,0.437788,0.043947
2,-2.539231,1.540688,-0.568637,0.501314,0.151070,0.263085,1.190944,-0.537757,-0.578083,1.492930,...,0.391203,-0.823968,-0.824352,0.158654,0.640648,0.018054,0.562864,0.678170,0.437788,0.490286
3,-2.532622,-0.575157,-0.691471,0.501314,0.962950,-0.198013,0.306142,-0.537757,-0.578083,-1.309804,...,-0.987486,-0.594053,0.241615,-0.022021,0.269135,-1.554868,0.562864,-0.551154,-0.970428,0.837439
4,-2.529221,1.540688,-1.182811,0.501314,0.348745,-0.124754,-0.357459,0.968527,-0.578083,-0.108632,...,-0.987486,1.347455,1.024223,-0.022021,0.680811,1.590976,-0.302485,0.678170,-0.970428,-0.228816
5,-2.528877,-0.575157,-1.244228,1.400808,1.284173,0.674626,0.859143,-0.537757,1.017574,-0.909413,...,0.391203,-0.517414,-1.404561,-0.383372,0.550280,0.542361,0.562864,1.907494,-0.970428,-0.898325
6,-2.527152,-0.575157,-1.336354,-0.398181,-0.470195,-0.025640,-0.468060,0.968527,-0.578083,-0.108632,...,1.769891,-0.721783,-1.350588,-0.383372,0.249054,0.018054,2.293562,0.678170,-0.970428,0.415896
7,-2.526131,1.540688,-0.507219,1.400808,1.559506,0.224301,0.527342,-0.537757,-0.578083,-0.909413,...,0.391203,0.836532,0.174149,1.122257,-1.578386,-0.506253,-0.302485,1.907494,-0.970428,-0.204019
8,-2.524383,-0.575157,-1.059976,1.400808,0.518181,-0.521213,-0.136259,-0.537757,1.017574,-0.509023,...,0.391203,0.402248,-1.431548,-0.262922,0.690852,-1.030561,-0.302485,1.907494,-0.970428,0.812642
9,-2.523090,1.540688,-1.121393,1.400808,0.737036,-0.189394,0.306142,-0.537757,1.017574,0.692149,...,-0.987486,-0.440776,1.078196,-0.202697,0.379585,-1.030561,-0.302485,-0.551154,-0.970428,0.564676


In [201]:
for C in [1.0, 2.0, 5.0, 10.0, 15.0, 20.0, 25.0]:
    
    start_time = datetime.datetime.now()
    
    lr = LogisticRegression(C=C, random_state=42, penalty='l2')
    
    lr.fit(X, y)
    
    auc = []
    
    cv_time = datetime.datetime.now()
    
    for train_index, test_index in kf.split(X):
    
        X_train = X[train_index]

        y_train = y[train_index]

        pred1 = lr.predict_proba(X_train)[:, 1]

        auc1 = roc_auc_score(y_train, pred1)
        
        auc.append(auc1)
        
    print("C: ", C, " Среднее значение AUC: ", round(np.mean(auc),4), " Время на кросс-валидацию: ", datetime.datetime.now() - cv_time, " Затраченное время: ", datetime.datetime.now() - start_time)

C:  1.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.412098  Затраченное время:  0:00:03.897024
C:  2.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.422123  Затраченное время:  0:00:03.983042
C:  5.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.408655  Затраченное время:  0:00:03.892026
C:  10.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.416642  Затраченное время:  0:00:03.910836
C:  15.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.412626  Затраченное время:  0:00:03.850390
C:  20.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.404615  Затраченное время:  0:00:03.933615
C:  25.0  Среднее значение AUC:  0.718  Время на кросс-валидацию:  0:00:00.406088  Затраченное время:  0:00:03.892530


In [202]:
lr = LogisticRegression(C=C, random_state=42, penalty='l2')
    
lr.fit(X, y)

LogisticRegression(C=25.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [203]:
pred1 = lr.predict_proba(X)[:, 1]

In [204]:
auc1 = roc_auc_score(y, pred1)

In [205]:
auc1

0.7179898314478335